## Installing Necessary Libraries

In [3]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 114.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

# Load Train Dataset

In [4]:
from datasets import load_dataset

# The instruction dataset to use
dataset_name = "koushik15042000/synthia_source_data"

# Load the dataset
dataset = load_dataset(dataset_name, split="train")

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

# Load Test Data

In [5]:
# Load the dataset
test_dataset = load_dataset(dataset_name, split="test")

Repo card metadata block was not found. Setting CardData to empty.


## Loading the model

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

Let's also load the tokenizer below

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

**Let's check what the base model predicts before finetuning. :)**

In [8]:
import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)


sequences = pipeline(
   ["You are a helpful agent that derives knowledge graph from the text : Angie Mentink (born October 18, 1972) is the co-anchor of Mariners Live. Mentink joined FSN in 1998, reporting on the Seattle Mariners and Seattle Seahawks for FSN Northwest. Prior to entering the broadcast profession, Mentink was a three-time college All-America softball player at the University of Washington and later played professional baseball as an original member of the Colorado Silver Bullets. Please create a Knowledge Graph in RDF Format, also give me Entities and Relationships",
    "You are a helpful agent that derives knowledge graph from the english text : Cary Granat is co-founder and CEO of Walden Media, a production company whose goal is to produce films that entertain and educate. Please create a Knowledge Graph in RDF Format, also give me Entities and Relationships"
    ],
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq[0]['generated_text']}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Result: You are a helpful agent that derives knowledge graph from the text : Angie Mentink (born October 18, 1972) is the co-anchor of Mariners Live. Mentink joined FSN in 1998, reporting on the Seattle Mariners and Seattle Seahawks for FSN Northwest. Prior to entering the broadcast profession, Mentink was a three-time college All-America softball player at the University of Washington and later played professional baseball as an original member of the Colorado Silver Bullets. Please create a Knowledge Graph in RDF Format, also give me Entities and Relationships for a given text.
The Knowledge Graph is a new search technology that provides a much richer, user-centric search experience. It does this by understanding real world entities and the relationships between those entities. This is done by using structured data to help the computer understand your intent.
What
Result: You are a helpful agent that derives knowledge graph from the english text : Cary Granat is co-founder and CEO of

Below we will load the configuration file in order to create the LoRA model. According to QLoRA paper, it is important to consider all linear layers in the transformer block for maximum performance.

In [9]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","v_proj"]
)

## Loading the trainer

Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below.

In [10]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 120
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

Then finally pass everthing to the trainer

In [11]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    # train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    # dataset_text_field="prediction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:247: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [12]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## Train the model

Now let's train the model! Simply call `trainer.train()`

In [13]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,1.505800
2,1.442200
3,1.546800
4,1.387500
5,1.361600
6,1.351800
7,1.244600
8,1.285400
9,1.134100
10,1.091300


TrainOutput(global_step=120, training_loss=0.3085162779626747, metrics={'train_runtime': 282.8169, 'train_samples_per_second': 6.789, 'train_steps_per_second': 0.424, 'total_flos': 2.793097966367539e+16, 'train_loss': 0.3085162779626747, 'epoch': 36.92})

In [14]:
lst_test_data_short = ["Live Johnny Winter And is a 1971 album by Johnny Winter recorded live at various locations during the fall of 1970 including the Capitol Theatre in Port Chester, New York, and Pirate's World in Dania, Florida. - Extract the Knowledge Graph",
 "Empedaula is a genus of moth in the family Gelechiidae. - Extract the Knowledge Graph",
 "Mei Li is a book by Thomas Handforth. Released by Doubleday, it was the recipient of the Caldecott Medal for illustration in 1939. - Extract the Knowledge Graph",
 "The Awful Battle of the Pekes and the Pollicles is a song in the musical Cats. It is sung mostly by Munkustrap. It describes a battle between two neighbouring dog tribes, the Pekes and the Pollicles, who bark until you could hear them all over the park. The cats stage the battles by dressing up as dogs and barking in each other\'s faces. This goes on until the Great Rumpus Cat arrives and scares the dogs away. - Extract the Knowledge Graph",
 "John Simon Bercow (born January 19 1963) is a politician and Conservative Party Member of Parliament for Buckingham in the United Kingdom with a current majority of 18,129 votes. He has a long-standing interest in Burma and has frequently raised issues of democracy and genocide in the country. In 2006 he was made a Patron of the Tory Reform Group. - Extract the Knowledge Graph",
 "Meetin\' WA is a 1986 short film by Jean-Luc Godard. In the film, he interviews his old friend Woody Allen. Throughout the film there are cuts in the interviews featuring photographs and film clips from Woody Allen\'s films. The two of them talk about movies, life, relationships to other directors and actors and Woody\'s past. The segments of the interview are separated by title cards, black cards and white type, a Woody Allen trademark. It runs for 26 minutes. - Extract the Knowledge Graph",
 "Tuek Thla is a khum (commune) of Serei Saophoan District in Banteay Meanchey Province in north-western Cambodia - Extract the Knowledge Graph",
 "Rosh HaNikra is a kibbutz in northern Israel. Located on the Mediterranean coast near the Rosh HaNikra grottoes and the border with Lebanon, it falls under the jurisdiction of Mateh Asher Regional Council. In 2006 it had a population of 746.- Extract the Knowledge Graph",
 "Human Frailty is an album released by Australian band Hunters & Collectors in 1986. It was their commercial breakthrough album in Australia, and a commercial and critical success. Singles from the album were 'Say Goodbye', 'Throw Your Arms Around Me' (a re-recorded version of the previous release as a single) and 'Everything's On Fire'. - Extract the Knowledge Graph",
 "Cognitive Ergonomics studies cognition in work settings, in order to optimize human well-being and system performance. It is a subset of the larger field of human factors and ergonomics.- Extract the Knowledge Graph",
 "Frieling USA is an American manufacturing company whose focus is on high-end kitchen products, serveware and coffee/tea accessories for home and commercial use.- Extract the Knowledge Graph"]

In [15]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
    lst_test_data_short,
    max_length=100,  #200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for ix,seq in enumerate(sequences):
    print(ix,seq[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 118, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 129, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


0 Live Johnny Winter And is a 1971 album by Johnny Winter recorded live at various locations during the fall of 1970 including the Capitol Theatre in Port Chester, New York, and Pirate's World in Dania, Florida. - Extract the Knowledge Graph from the english text provided by the user. Winter had recently released his third solo album, Still Alive and Well, when he recorded and released this live album in 1971
1 Empedaula is a genus of moth in the family Gelechiidae. - Extract the Knowledge Graph triples from the english text provided by the user. Entries are derived from the Coordinator, a knowledge graph developed by the BBC, deriving from the user-entered text provided by the user. The user-entered text is given in the form of a series of noun-relationship-object triples, i.e. the user-
2 Mei Li is a book by Thomas Handforth. Released by Doubleday, it was the recipient of the Caldecott Medal for illustration in 1939. - Extract the Knowledge Graph from the text provided by the user. -